# Отчет. Aнализ результатов A/B - теста для величины, имеющей распределение Бернулли

In [1]:
# необходимые импорты

import pandas as pd
from scipy.stats import norm
from scipy.stats import pointbiserialr
from scipy.stats import anderson_ksamp
from scipy.stats import ks_2samp
from statsmodels.stats.proportion import proportions_ztest

### 1. Определение целевой метрики.

Согласно условию проекта целью проводимого теста
является установление влияния нового процесса авторизации на частоту утилизаций счёта. Поэтому в качестве целевой метрики выберем утилизации счёта (в исторических данных они представлены столбцом "**Флаг утилизации счёта**").

Данная метрика имеет распределение Бернулли.

В качестве агрегирующей оценки целевой метрики будем использовать конверсию в утилизации счета (отношение количества утилизированных счетов к общему количеству счетов). Обозначим данную оценку как **conv**. Рассчитаем ее величину для исторических данных, то есть найдем **base_conv**, от которой далее будем отталкиваться при планировании эксперимента

In [2]:
# загружаем исторические данные
df_2 = pd.read_csv('hist_credit_card.csv', header=0, sep=',')
df_2.head()

,ID,Возраст,Доход клиента,Вероятность банкротства,Флаг утилизации счёта,Расходы,PV КК,PV услуги,NPV
0,0,19,21620.835463,0.138061,0,102,0,0,-102
1,1,27,24897.990863,0.035508,1,409,11686,1754,13031
2,2,50,23989.526947,0.098793,0,16,0,0,-16
3,3,18,38442.409756,0.365661,1,788,13738,1578,14528
4,4,24,21291.521612,0.036909,1,1048,6594,2213,7759


In [3]:
# рассчитываем baseline conversion
base_conv = round(df_2['Флаг утилизации счёта'].sum() / df_2['Флаг утилизации счёта'].count(), 3) * 100
print(f'Конверсия в утилизации счета на исторических данных = {base_conv} %')

Конверсия в утилизации счета на исторических данных = 74.2 %


### 2. Нулевая гиппотеза и альтернатива.

Введем обозначения: р1 - частота утилизации счета на тестовой выборке, р2 - частота утилизации счета на контрольной выборке.

Нулевая гипотеза Н0: p1 = p2 (то есть внедрение нового процесса авторизации не окажет статистически значимого влияния на частоту утилизации счёта)

Альтернатива Н1: p1 != р2 (внедрение нового процесса авторизации повлияет на частоту утилизации счёта)

Рассмотрим в данном случае двустороннюю альтернативу, так как нет однозначной уверенности, каким будет (и будет ли оно) влияние нового процесса авторизации на утилизации счёта. В случае, если нулевая гиппотеза будет отклонена, направление влияния (уменьшение/увеличение) нового процесса авторизации на утилизации счета может быть оценено на основании статистических характеристик контрольной и тестовой выборок

### 3. Определение параметров, влияющих на целевую метрику

Поскольку в рассматриваемом проекте мы ищем связь между бинарным ('Флаг утилизации счёта') и непрерывными ('Вероятность банкротства', 'Расходы', 'Возраст') показателями, то будем рассчитывать **точечно-биссериальный коэффициент корреляции**. Он представляет собой частный случай коэффициента корреляции Пирсона и позволяет установить связь между переменными вышеуказанного типа.

In [4]:
properties = ['Возраст', 'Доход клиента', 'Вероятность банкротства']
for i in properties:
  a = round(pointbiserialr(df_2['Флаг утилизации счёта'].values, df_2[i].values)[0], 5)
  print(f'Коэффициент корреляции с \'{i}\' = {a}')

Коэффициент корреляции с 'Возраст' = 0.00728
Коэффициент корреляции с 'Доход клиента' = 0.00071
Коэффициент корреляции с 'Вероятность банкротства' = -0.2154


Из полученных результатов видно, что корредяция целевой метрики с переменными 'Возраст' и 'Доход клиента' практически отсутствует. Принимая во внимание достаточно большой размер выборки исторических данных, на котором исследовалась корреляция переменных, можем отметить значимую обратную корреляцию целевой метрики с величиной 'Вероятность банкротства'.

### 4. Расчет размера выборки

Размер выборки считаем по формуле

$$ n = m = \frac{2 p_{0} (1 - p_{0}) (z_{1-\alpha} - z_{\beta})^2}{d^2} $$

In [5]:
p0 = base_conv / 100

alpha = 0.05
z_alpha = norm.ppf(1 - alpha)
betta = 0.05   # по условию мы хотели бы различать 3%-ное изменение целевой метрики с вероятностью 95%, т.е. 1- betta = 95/100
z_betta = norm.ppf(betta)
d = 0.03 * df_2['Флаг утилизации счёта'].values.mean()  # mde, равное uplift(3% из условия), умноженное на среднее(из исторических данных)

n = 2*p0*(1-p0)*((z_alpha - z_betta)**2) / (d**2)
print(f'Минимальный размер выборки = {n}')

Минимальный размер выборки = 8372.435417125778


Задаем размер выборки N = 8373

### 5. Анализ результатов A/B теста

### 5.1. Проверка однородности контрольной и тестовой выборок

Проверяем однородность тестовой и контрольной выборок по параметру "Вероятность банкротства" (как параметру, с которым установленая обратная корреляция целевой метрики). Для проверки однородности используем двухвыборочный критерий Андерсона-Дарлинга.

In [6]:
# загружаем контрольную и тестовую выборки а формируем вектора из величин столбцов "Вероятность банкротства"
df_control = pd.read_csv('control.csv', header=0, sep=',')
df_test = pd.read_csv('test.csv', header=0, sep=',')

bankrupcy_control = df_control['Вероятность банкротства'].values
bankrupcy_test = df_test['Вероятность банкротства'].values

# рассчитываем показатели критерия
p1 = anderson_ksamp([bankrupcy_test, bankrupcy_control])
print(p1)

Anderson_ksampResult(statistic=-0.36404574150101787, critical_values=array([0.325, 1.226, 1.961, 2.718, 3.752, 4.592, 6.546]), pvalue=0.25)


C:\Users\alena\AppData\Local\Temp\ipykernel_15256\833448056.py:9: UserWarning: p-value capped: true value larger than 0.25
  p1 = anderson_ksamp([bankrupcy_test, bankrupcy_control])


Для нашего случая данный критерий не сработал. Он выдал предупреждение, что p-значение (25%) было ограничено и может быть не очень точным. Поэтому воспользуемся для оценки однородности контрольной и тестовой выборок (быть может не таким эффективным, но вполне применимым) критерием Колмагорова-Смирнова

In [7]:
p1 = ks_2samp(bankrupcy_control, bankrupcy_test)
print(f'Результаты расчета критерия: \nстатистика = {p1[0]}, р-значение = {p1[1]}')


Результаты расчета критерия: 
статистика = 0.014092917711692344, р-значение = 0.37658779385844954


Нулевая гиппотеза для критерия однородности состоит в том, что выборки однородны. Она может быть отклонена, если р-значение, полученное в результате расчета критерия, выйдет за границы интервала [alpha/2, 1 - alpha/2], где alpha - уровень значимости. Согласно условию проекта, проверка однородности выбранного параметра осуществяется с уровнем значимости 2%. Таким образом, оснований для отклонения гипотезы об однородности тестовой и контрольной выборок нет.


### 5.2. Принятие решения 

Поскольку в данном задании мы проверяем, соотношение частот для тестовой и контрольной выборок, то используем критерий proportions_ztest

In [8]:
# формируем массивы из данных об утилизации счета для тестовой и контрольной выборок
ut_test = df_test['Флаг утилизации счёта'].values
ut_control = df_control['Флаг утилизации счёта'].values

# подсчитываем количество утилизаций счета для тестовой и контрольной выборок
test_success = ut_test.sum()
control_success = ut_control.sum()

# количество элементов в выборках
n = ut_test.shape[0]    

# расчитываем статистику и р-значение критерия
p = proportions_ztest([test_success, control_success],[n, n], alternative='two-sided')
print(f'Результаты расчета критерия: \nстатистика = {p[0]}, р-значение = {p[1]}')

Результаты расчета критерия: 
статистика = -0.6341302274915823, р-значение = 0.5259958373299385


Из полученных результатов расчета z-теста видим, что р-значение попало в интервал [alpha/2, 1-alpha/2], где уровень значимости alpha=0.05.

Таким образом, при статистических вводных, заданных в условии теста, у нас нет оснований отклонять нулевую гиппотезу о том, что внедрение нового процесса авторизации не окажет статистически значимого влияния на частоту утилизации счёта.